In [1]:
import torch
import random
import numpy as np
from functools import partial

In [2]:
# pip install torch torch_geometric torch-scatter pandas scikit-learn wandb
# python -m pip install gurobipy

In [3]:
from temp.data.gen import parallel_generate_problem, parallel_generate_solutions
from temp.data.problem import setcover
from temp.data.info import ModelInfo
from temp.data.dataset import ModelGraphDataset
from temp.data.aug import parallel_augment_info, augment_info


from temp.solver.utils import solve_inst


In [4]:
PRE_TRAIN_DIR = "temp/pre_train"
TRAIN_DIR = "temp/train"
VALID_DIR = "temp/valid"

In [5]:
# parallel_generate_problem(setcover, PRE_TRAIN_DIR, 100, 10)
# parallel_generate_solutions(PRE_TRAIN_DIR, 10)

# parallel_generate_problem(setcover, TRAIN_DIR, 10000, 10)
# parallel_generate_solutions(TRAIN_DIR, 10)

# parallel_generate_problem(setcover, VALID_DIR, 100, 10)
# parallel_generate_solutions(VALID_DIR, 10)

In [6]:
m = setcover()
vals = solve_inst(m)
info = ModelInfo.from_model(m)
info.var_info.sols = np.array([[m.objVal] + vals])

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 128 rows, 256 columns and 3285 nonzeros
Model fingerprint: 0x3820ea4a
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 216.0000000
Presolve time: 0.04s
Presolved: 128 rows, 256 columns, 3285 nonzeros
Variable types: 0 continuous, 256 integer (256 binary)

Root relaxation: objective 7.016822e+01, 213 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Nod

In [7]:
# from temp.data.dataset import info_to_data
# d = info_to_data(info)
# aug = partial(parallel_augment_info, n=100)

In [8]:
pre_train_d = ModelGraphDataset(PRE_TRAIN_DIR, augment=augment_info)
train_d = ModelGraphDataset(TRAIN_DIR, augment=augment_info)
valid_d = ModelGraphDataset(VALID_DIR)

data = pre_train_d[0][1]
var_feature_size = data.var_node_features.size(-1)
con_feature_size = data.con_node_features.size(-1) 

In [9]:
import pandas as pd
CFG = pd.read_excel("temp/trained_models/setcover_model_configs.xlsx", index_col=0).loc[0].to_dict()
CFG["num_epochs"] = 5
CFG["num_layers"] = 8
CFG["hidden"] = 8

In [10]:
from temp.model.utils import get_model
from temp.model.trainer import train

In [11]:
model_name, model, criterion, optimizer, scheduler = get_model(".", var_feature_size, con_feature_size, n_batches=1, **CFG)

In [12]:
scheduler.total_steps = 10000

In [13]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed()
    np.random.seed(worker_seed)
    random.seed(worker_seed)

from torch_geometric.loader import DataLoader
pretrain_loader = DataLoader(pre_train_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
train_loader = DataLoader(train_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))
val_loader = DataLoader(valid_d, batch_size=8, shuffle=True, worker_init_fn=seed_worker, generator=torch.Generator().manual_seed(0))

In [14]:
scheduler.total_steps = 1000000

In [15]:
train(model_name, model, criterion, optimizer, scheduler, pretrain_loader, train_loader, val_loader, CFG, False, "./")

>> Training starts on the current device cpu
>> Pretraining for prenorm...


  0%|          | 0/13 [00:00<?, ?it/s]


>> Epoch 1 ----------------------------------------------------------------------------------------------------
Training... 0


 10%|█         | 131/1250 [02:28<18:53,  1.01s/it]

>>> tensor(1.)


...

val_loss                 0.269169
val_acc                  0.933850
val_f1                   0.321190
val_precision            0.526050
val_recall               0.231167
val_evidence_succ       15.130222
val_evidence_fail        4.686167
val_uncertainty_succ     0.150822
val_uncertainty_fail     0.387273
val_true_bias            0.067644
val_pred_bias            0.029760
val_soft_pred_bias       0.031571
val_bias_error           0.037885
val_lr                   0.000989
dtype: float32
>> Epoch 4 ----------------------------------------------------------------------------------------------------
Training... 3750


  6%|▌         | 78/1250 [00:39<08:03,  2.42it/s]

>>> tensor(1.)


 41%|████      | 514/1250 [04:58<07:08,  1.72it/s]


KeyboardInterrupt: 

In [16]:
from temp.data.dataset import info_to_data

In [17]:
m = setcover(512, 256)

In [18]:
info = ModelInfo.from_model(m)
data = info_to_data(info)

In [19]:
with torch.no_grad():
    logits = model(data)

In [20]:
binary_mask = torch.ones(len(logits), dtype=bool)

In [21]:
from temp.solver import upr

In [22]:
probs, preds = upr.get_predictions(logits, binary_mask)

In [23]:
unc = upr.get_uncertainty(logits)

In [24]:
threshold = upr.get_threshold(unc)

In [25]:
import json
from tempfile import NamedTemporaryFile

import gurobipy as gp


def fix_var(inst, idxs, vals):
    assert len(idxs) == len(vals)
    bounds = {}
    vs = inst.getVars()
    for idx, val in zip(idxs, vals):
        v = vs[idx]
        bounds[idx] = (v.lb, v.ub)
        v.setAttr("lb", val)
        v.setAttr("ub", val)
    inst.update()
    return bounds


def unfix_var(inst, idxs, bounds):
    assert len(idxs) == len(bounds)
    vs = inst.getVars()
    print(idxs, bounds)
    for i, (lb, ub) in zip(idxs, bounds):
        vs[i].setAttr("lb", lb)
        vs[i].setAttr("ub", ub)


def get_iis_vars(inst):
    try:
        inst.computeIIS()
    except Exception as e:
        print(e)
        if "Cannot compute IIS on a feasible model" in str(e):
            return set()
        raise e

    with NamedTemporaryFile(suffix=".ilp", mode="w+") as f:
        inst.write(f.name)
        f.seek(0)
        return set(f.read().split())


def set_starts(inst, starts):
    vs = inst.getVars()
    for i, s in starts.items():
        vs[i].setAttr("lb", s)


def solve_inst(inst):
    vs = inst.getVars()
    inst.optimize()
    return inst.getAttr("X", vs)


def repair(inst, fixed: set, bounds: dict):
    old_iis_method = getattr(inst, "IISMethod", -1)
    inst.setParam("IISMethod", 0)

    vs = inst.getVars()
    ns = inst.getAttr("varName", vs)
    name_to_idx = {n: i for i, n in enumerate(ns)}

    freed = set()
    while iis_var_names := get_iis_vars(inst):
        for n in iis_var_names:
            if n not in name_to_idx:
                continue

            var_idx = name_to_idx[n]
            if var_idx not in fixed:
                continue

            if var_idx in freed:
                continue

            lb, ub = bounds[var_idx]
            vs[var_idx].lb = lb
            vs[var_idx].ub = ub
            freed.add(var_idx)

    inst.setParam("IISMethod", old_iis_method)
    return freed


def with_lic(m, path="gb.lic"):
    with open(path) as f:
        env = gp.Env(params=json.load(f))
    return m.copy(env=env)


In [26]:
import numpy as np
import torch
import torch.nn.functional as F

# from temp.solver.utils import fix_var, repair, set_starts, unfix_var, solve_inst

EVIDENCE_FUNCS = {
    "softplus": (lambda y: F.softplus(y)),
    "relu": (lambda y: F.relu(y)),
    "exp": (lambda y: torch.exp(torch.clamp(y, -10, 10))),
}


def get_predictions(logits, binary_mask):
    probs = torch.softmax(logits, axis=1)
    preds = probs[:, 1]
    probs = _to_numpy(probs)
    preds = _to_numpy(preds).squeeze()
    preds[binary_mask] = preds[binary_mask].round()
    return probs, preds


def get_uncertainty(logits, evidence_func_name: str = "softplus"):
    evidence = EVIDENCE_FUNCS[evidence_func_name](logits)
    alpha = evidence + 1
    uncertainty = logits.shape[1] / torch.sum(alpha, dim=1, keepdim=True)
    return uncertainty


def get_threshold(uncertainty: torch.Tensor, r_min: float = 0.4, r_max: float = 0.55):
    q = (r_min + r_max) / 2
    threshold = torch.quantile(uncertainty, q)
    r = (uncertainty <= threshold).float().mean()

    if r > r_max:
        threshold = torch.quantile(uncertainty, r_max)
        return threshold

    if r < r_min:
        threshold = torch.quantile(uncertainty, r_min)
        return threshold

    return threshold


def get_confident_idx(indices, uncertainty, threshold):
    confident_mask = uncertainty <= threshold
    confident_idx = indices[confident_mask]
    return confident_idx.sort()[0]


def solve(inst, prediction, uncertainty, indices, max_iter):
    threshold = get_threshold(uncertainty)
    conf_idxs = get_confident_idx(indices, uncertainty, threshold)
    conf_vals = prediction[conf_idxs]
    bounds = fix_var(inst, conf_idxs, conf_vals)
    print(len(bounds), len(prediction))

    min_q = sum(uncertainty <= threshold) / len(uncertainty)
    max_q = 1.0
    dq = (max_q - min_q) / (max_iter - 1)

    fixed = set(conf_idxs.tolist())
    freed = set(repair(inst, fixed, bounds))
    for i in range(1, max_iter):
        sol = solve_inst(inst)
        q = max_q - dq * i
        threshold = np.quantile(uncertainty, q)
        conf_idxs = get_confident_idx(indices, uncertainty, threshold)
        to_unfix = list(fixed - set(conf_idxs.tolist()))
        to_unfix = [i for i in to_unfix if i not in freed]
        unfix_var(inst, to_unfix, [bounds[i] for i in to_unfix])
        starts = {i: sol[i] for i in to_unfix}
        starts.update({i: sol[i] for i in freed})
        set_starts(inst, starts)
    return sol


def _to_numpy(tensor_obj):
    return tensor_obj.cpu().detach().numpy()


In [27]:
indices = torch.arange(0, len(unc), dtype=int)
unc = unc.squeeze()
to_solve = m.copy()
to_solve.setParam("TimeLimit", 20)
solve(to_solve, preds, unc, indices, 5)

Set parameter TimeLimit to value 20
122 256
Set parameter IISMethod to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  20
IISMethod  0

IIS computation: initial model status unknown, solving to determine model status
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
IIS runtime: 0.01 seconds (0.00 work units)
Cannot compute IIS on a feasible model
Set parameter IISMethod to value -1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GH

[0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [41]:
gb_m = m.copy()
gb_m.setParam("TimeLimit", 150)
gb_m.optimize()

Set parameter TimeLimit to value 150
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  150

Optimize a model with 512 rows, 256 columns and 13120 nonzeros
Model fingerprint: 0x7090591e
Variable types: 0 continuous, 256 integer (256 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 299.0000000
Presolve time: 0.01s
Presolved: 512 rows, 256 columns, 13120 nonzeros
Variable types: 0 continuous, 256 integer (256 binary)

Root relaxation: objective 9.193545e+01, 578 iterations, 0.04 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap